In [100]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [101]:
import pandas as pd
import numpy as np
import re
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from datetime import datetime
import ast

In [102]:
metadata_df = pd.read_csv('/content/drive/MyDrive/metavis/socrata_nycopendata_metadata_fetched_20211105.csv')
metadata_df.head()

,data_uuid,approvals,assetType,attribution,attributionLink,averageRating,blobFileSize,blobFilename,blobId,blobMimeType,category,childViews,columns,createdAt,data_page_url,description,disabledFeatureFlags,displayFormat,displayType,downloadCount,flags,grants,hideFromCatalog,hideFromDataJson,iconUrl,indexUpdatedAt,license,licenseId,locale,metadata,modifyingViewUid,name,newBackend,numberOfComments,oid,owner,previewImageId,provenance,publicationAppendEnabled,publicationDate,publicationGroup,publicationStage,query,ratings,rights,rowClass,rowIdentifierColumnId,rowsUpdatedAt,rowsUpdatedBy,tableAuthor,tableId,tags,totalTimesRated,viewCount,viewLastModified,viewType
0,2232-dj5q,"[{'reviewedAt': 1543956693, 'reviewedAutomatic...",dataset,"Department of Youth and Community Development,...",https://www1.nyc.gov/assets/operations/downloa...,0,NaN,NaN,NaN,NaN,Social Services,NaN,"[{'id': 386547993, 'name': 'category', 'dataTy...",1543949705,https://data.cityofnewyork.us/Social-Services/...,Data on the youth who are currently residing i...,NaN,NaN,table,1057,"['default', 'ownerMayBeContacted', 'restorable...","[{'inherited': False, 'type': 'viewer', 'flags...",False,False,NaN,1.564583e+09,NaN,NaN,NaN,{'jsonQuery': {'order': [{'columnFieldName': '...,NaN,Local Law 37 - DYCD Report,True,0,30280661,"{'id': '5fuc-pqz2', 'displayName': 'NYC OpenDa...",NaN,official,False,1543949887,15737765,published,"{'orderBys': [{'ascending': True, 'expression'...",NaN,['read'],NaN,NaN,1.632316e+09,d5dp-fses,"{'id': '5fuc-pqz2', 'displayName': 'NYC OpenDa...",15737765,"['runaway and homeless youth', 'temporary and ...",0,924,1632316033,tabular
1,22gm-5ceg,"[{'reviewedAt': 1608131640, 'reviewedAutomatic...",map,Mayor's Office of Environmental Remediation (M...,NaN,0,NaN,NaN,NaN,NaN,Environment,NaN,[],1606245240,https://data.cityofnewyork.us/Environment/Shap...,GIS shapefiles showing the areas in NYC where ...,NaN,{'visualizationCanvasMetadata': {'vifs': [{'co...,visualization_canvas_map,0,['ownerMayBeContacted'],"[{'inherited': True, 'type': 'viewer', 'flags'...",False,False,NaN,NaN,NaN,NaN,NaN,"{'rdfClass': '', 'rdfSubject': '0', 'attachmen...",w5ew-m3sm,Shapefiles of BOA and Community Brownfield Pla...,True,0,35839269,"{'id': '5fuc-pqz2', 'displayName': 'NYC OpenDa...",NaN,official,False,1606245568,17806672,published,{},NaN,['read'],NaN,NaN,1.606166e+09,d5dp-fses,"{'id': '5fuc-pqz2', 'displayName': 'NYC OpenDa...",17806672,"['moer', 'oer', 'boa']",0,201,1608131673,tabular
2,22rr-ujq3,"[{'reviewedAt': 1528410117, 'reviewedAutomatic...",dataset,NYC Department of Education,NaN,0,NaN,NaN,NaN,NaN,Education,NaN,"[{'id': 362632320, 'name': 'DBN', 'dataTypeNam...",1528409339,https://data.cityofnewyork.us/Education/2016-2...,Student discipline annual report for English l...,NaN,NaN,table,287,"['default', 'ownerMayBeContacted', 'restorable...","[{'inherited': False, 'type': 'viewer', 'flags...",False,False,NaN,1.534454e+09,NaN,NaN,en_US,"{'rdfClass': '', 'rdfSubject': '0', 'attachmen...",NaN,2016-2017 Student Discipline Annual Report - ELL,True,0,28988615,"{'id': 'hecw-wydt', 'displayName': 'Annette', ...",NaN,official,False,1528410117,15258121,published,{},NaN,['read'],NaN,NaN,1.528409e+09,hecw-wydt,"{'id': 'hecw-wydt', 'displayName': 'Annette', ...",15258121,"['removals', 'expulsions', 'ell (english langu...",0,197,1625100154,tabular
3,23z9-6uk9,"[{'reviewedAt': 1572275203, 'reviewedAutomatic...",dataset,Department of Education (DOE),NaN,0,NaN,NaN,NaN,NaN,Education,NaN,"[{'id': 434884014, 'name': 'dbn', 'dataTypeNam...",1571867795,https://data.cityofnewyork.us/Education/2020-D...,Directory of Department of Education High Scho...,NaN,NaN,table,1094,"['default', 'ownerMayBeContacted', 'restorable...","[{'inherited': False, 'type': 'viewer', 'flags...",False,False,NaN,1.572275e+09,NaN,NaN,NaN,"{'rdfClass': '', 'rdfSubject': '0', 'attachmen...",NaN,2020 DOE High School Directory,True,0,32699286,"{'id': '5fuc-pqz2', 'displayName': 'NYC OpenDa...",NaN,official,False,15718689

In [4]:
metadata_df.shape

(3405, 56)

Check missing data

In [5]:
metadata_df.isnull().sum()/metadata_df.shape[0]

data_uuid                   0.000000
approvals                   0.000000
assetType                   0.000000
attribution                 0.062261
attributionLink             0.895448
averageRating               0.000000
blobFileSize                0.873421
blobFilename                0.873421
blobId                      0.873421
blobMimeType                0.873421
category                    0.025844
childViews                  0.929222
columns                     0.000000
createdAt                   0.000000
data_page_url               0.000000
description                 0.036417
disabledFeatureFlags        0.987078
displayFormat               0.888106
displayType                 0.000000
downloadCount               0.000000
flags                       0.000000
grants                      0.000000
hideFromCatalog             0.000000
hideFromDataJson            0.000000
iconUrl                     0.986490
indexUpdatedAt              0.273128
license                     0.976505
l

In [6]:
metadata_df['attribution'].value_counts()

NYC Department of Education                                                                                                                                                          471
Department of Education (DOE)                                                                                                                                                        258
Department of Parks and Recreation (DPR)                                                                                                                                             173
Department of Transportation (DOT)                                                                                                                                                   166
Department of Education                                                                                                                                                              165
                                                                           

In [7]:
metadata_df['assetType'].value_counts()

dataset          2612
map               349
file              190
href              188
datalens           32
filter             30
visualization       3
chart               1
Name: assetType, dtype: int64

In [8]:
metadata_df['averageRating'].value_counts()

0      3371
100      17
20       10
80        5
40        2
Name: averageRating, dtype: int64

In [9]:
metadata_df['category'].value_counts()

Education                1110
City Government           885
Transportation            275
Social Services           249
Housing & Development     207
Environment               189
Public Safety             129
Recreation                 84
Health                     78
Business                   75
NYC BigApps                36
Name: category, dtype: int64

In [10]:
metadata_df['displayType'].value_counts()

table                         2642
map                            267
blob                           190
href                           188
visualization_canvas_map        82
data_lens                       32
visualization_canvas_table       3
visualization_canvas_chart       1
Name: displayType, dtype: int64

In [103]:
#feature_df = metadata_df.drop(metadata_df.columns.difference(['data_uuid','assetType', 'attribution','averageRating','category','createdAt','displayType','downloadCount','flags',
#                                                              'numberOfComments','newBackend','metadata','provenance','publicationAppendEnabled',
#                                                              'rowsUpdatedAt',"rowsUpdatedBy",'viewCount','viewType']), 1)

feature_df = metadata_df.drop(metadata_df.columns.difference(['data_uuid','assetType', 'attribution','category','createdAt','displayType','flags',
                                                              'newBackend','metadata','provenance','publicationAppendEnabled',
                                                              'rowsUpdatedAt',"rowsUpdatedBy",'viewType']), 1)


In [104]:
# Extract create date
feature_df['createDate'] = feature_df['createdAt'].apply(lambda x: datetime.fromtimestamp(x))
feature_df['createYear'] = pd.DatetimeIndex(feature_df['createDate']).year
feature_df['createMonth'] = pd.DatetimeIndex(feature_df['createDate']).month

# Extract dataset age
diff = []
today = datetime.today()
for i in feature_df['createDate']:
  d = today-i
  day_diff = d.days
  if day_diff <0:
    diff.append(-1)
  else:
    diff.append(day_diff)
feature_df['createAtAge'] = diff

feature_df = feature_df.drop(['createDate','createdAt'], axis = 1)

In [105]:
# extract flags
#['default', 'ownerMayBeContacted', 'restorable', 'restorePossibleForType']    2612
#['default', 'ownerMayBeContacted']                                             619
#['ownerMayBeContacted']                                                        144
#['ownerMayBeContacted', 'restorable', 'restorePossibleForType']                 30

feature_df['flags_default'] = feature_df['flags'].apply(lambda x: 1 if 'default' in x else 0)
feature_df['flags_ownerMayBeContacted'] = feature_df['flags'].apply(lambda x: 1 if 'ownerMayBeContacted' in x else 0)
feature_df['flags_restorable'] = feature_df['flags'].apply(lambda x: 1 if 'restorable' in x else 0)
feature_df['flags_restorePossibleForType'] = feature_df['flags'].apply(lambda x: 1 if 'restorePossibleForType' in x else 0)
feature_df = feature_df.drop(['flags'], axis = 1)

In [106]:
# Extract tableAuthor-screenName
feature_df['tableAuthor_clean'] = metadata_df['tableAuthor'].apply(lambda x: ast.literal_eval(x))

l = []
for i in feature_df['tableAuthor_clean']:
  l.append(i['screenName'])

feature_df['tableAuthor_screenName'] = feature_df['tableAuthor_clean'].apply(lambda x: x['screenName'])

In [107]:
# Extract tableAuthor-flags
acceptedEula_l = []
mayBeStoriesCoOwner_l = []
for i in feature_df['tableAuthor_clean']:
  try:
    if 'acceptedEula' in i['flags']:
      acceptedEula_l.append(1)
    else:
      acceptedEula_l.append(0)
  except:
    acceptedEula_l.append(0)

  try:
    if 'mayBeStoriesCoOwner' in i['flags']:
      mayBeStoriesCoOwner_l.append(1)
    else:
      mayBeStoriesCoOwner_l.append(0)
  except:
    mayBeStoriesCoOwner_l.append(0)

feature_df['tableAuthor_flags_acceptedEula'] = acceptedEula_l
feature_df['tableAuthor_flags_mayBeStoriesCoOwner'] = mayBeStoriesCoOwner_l

feature_df = feature_df.drop(['tableAuthor_clean'], axis = 1)

In [108]:
# Extract rowsUpdatedAt 
from datetime import datetime
l = []
for i in feature_df['rowsUpdatedAt']:
  try:
    l.append(datetime.fromtimestamp(i))
  except:
    l.append(datetime(2022, 1, 1))

feature_df['rowsUpdatedAtDate'] = l
feature_df['rowsUpdatedAtYear'] = pd.DatetimeIndex(feature_df['rowsUpdatedAtDate']).year
feature_df['rowsUpdatedAtMonth'] = pd.DatetimeIndex(feature_df['rowsUpdatedAtDate']).month

diff = []
today = datetime.today()
for i in feature_df['rowsUpdatedAtDate']:
  d = today-i
  day_diff = d.days
  if day_diff <0:
    diff.append(-1)
  else:
    diff.append(day_diff)
feature_df['rowsUpdatedAtAge'] = diff
feature_df = feature_df.drop(['rowsUpdatedAtDate','rowsUpdatedAt'], axis = 1)

In [109]:
 feature_df['metadata_clean'] = feature_df['metadata'].apply(lambda x: ast.literal_eval(x))

In [110]:
# extract metadata-Agency, which can be used as a broader feature of attribution. 
feature_df['metadata_Agency'] = feature_df['metadata_clean'].apply(lambda x: x['custom_fields']['Dataset Information']['Agency'] if ('custom_fields' in x.keys()) and ('Dataset Information' in x['custom_fields'].keys()) else "Unknown")
agency_count = list(zip(feature_df['metadata_Agency'].value_counts(),feature_df['metadata_Agency'].value_counts().index))
s = 0
main_agency = []
for i in agency_count:
  s += i[0]
  main_agency.append(i[1])
  if s > feature_df.shape[0]*0.9:
    break
# print(len(main_agency)) --> 35
# Group other categories into Other
feature_df['metadata_AgencyClean'] = feature_df['metadata_Agency'].apply(lambda x: x if x in main_agency else 'Other' )
feature_df = feature_df.drop(['metadata_Agency','attribution'], axis = 1)

In [111]:
# extract metadata-Update Frequency
l_UpdateFrequency = []
for i in feature_df['metadata_clean']:
  try:
    displaytype =i['custom_fields']['Update']['Update Frequency']
  except:
    displaytype = []
  if type(displaytype) == list:
    l_UpdateFrequency.extend(displaytype)
  else:
    l_UpdateFrequency.append(displaytype)
#unique_l_availableDisplayTypes = set(l_availableDisplayTypes)
unique_l_UpdateFrequency = list(set(l_UpdateFrequency))
print(unique_l_UpdateFrequency)
feature_df['metadata_UpdateFrequency'] = feature_df['metadata_clean'].apply(lambda x: x['custom_fields']['Update']['Update Frequency'] if 'custom_fields' in x.keys() else [] )

['To Be Determined', 'As needed', 'Monthly', 'Triannually', 'Historical Data', 'Annually', 'Every six months', 'Every four years', 'Hourly', 'Weekdays', 'Weekly', 'Quarterly', 'Biannually ', '2 to 4 times per year', 'Biennially', 'Several times per day', 'Biweekly ', 'Daily', 'Bimonthly ']


In [117]:
# Update index_uuid_df by request


index_uuid_df_update = pd.DataFrame(metadata_df[['data_uuid','name','attribution', 'viewCount','downloadCount','description']])
index_uuid_df_update['updateFrequency'] = feature_df['metadata_UpdateFrequency']

l = []
for i in metadata_df['rowsUpdatedAt']:
  try:
    l.append(datetime.fromtimestamp(i))
  except:
    l.append(None)

index_uuid_df_update['rowsUpdatedAtDate'] = l

index_uuid_df_update = index_uuid_df_update.reset_index()
index_uuid_df_update.head()



,index,data_uuid,name,attribution,viewCount,downloadCount,description,updateFrequency,rowsUpdatedAtDate
0,0,2232-dj5q,Local Law 37 - DYCD Report,"Department of Youth and Community Development,...",924,1057,Data on the youth who are currently residing i...,Monthly,2021-09-22 13:07:15
1,1,22gm-5ceg,Shapefiles of BOA and Community Brownfield Pla...,Mayor's Office of Environmental Remediation (M...,201,0,GIS shapefiles showing the areas in NYC where ...,Annually,2020-11-23 21:15:43
2,2,22rr-ujq3,2016-2017 Student Discipline Annual Report - ELL,NYC Department of Education,197,287,Student discipline annual report for English l...,Historical Data,2018-06-07 22:09:08
3,3,23z9-6uk9,2020 DOE High School Directory,Department of Education (DOE),4443,1094,Directory of Department of Education High Scho...,Historical Data,2019-10-23 22:15:58
4,4,242c-ru4i,DSNY Special Waste Drop-off Sites,Department of Sanitation,388,596,Location of DSNY Special Waste Drop-Off Sites....,Monthly,2021-11-08 11:09:27


In [118]:
index_uuid_df_update.to_csv("/content/drive/MyDrive/metavis/index_uuid_df_update.csv")

In [20]:
# Ordinal Encoding on Update Frequency
def UpdateFrequency_encode(val):
    if val == 'Hourly':
      return 1
    elif val == 'Several times per day':
        return 2
    elif val == 'Every four years':
        return 3
    elif val == 'Daily':
        return 4
    elif val == 'Weekdays':
        return 5
    elif val == 'Weekly':
        return 6
    elif val == 'Biweekly ':
        return 7
    elif val == 'Bimonthly ':
        return 8
    elif val == 'Monthly':
        return 9
    elif val == 'Quarterly':
        return 10
    elif val == '2 to 4 times per year':
        return 11
    elif val == 'Triannually':
        return 12
    elif val == 'Biannually ':
        return 13
    elif val == 'Every six months':
        return 13
    elif val == 'Annually':
        return 14
    elif val == 'Biennially':
        return 15
    elif val == 'As needed':
        return 16
    elif val == 'Historical Data':
        return 17
    elif val == 'To Be Determined':
        return 18
    else:
      return 0
feature_df['metadata_UpdateFrequency_encode'] = feature_df['metadata_UpdateFrequency'].map(lambda x:UpdateFrequency_encode(x))
feature_df = feature_df.drop(['metadata_UpdateFrequency'], axis = 1)

In [21]:
# extract metadata-availableDisplayTypes

l_availableDisplayTypes = []
for i in feature_df['metadata_clean']:
  try:
    displaytype = i['availableDisplayTypes']
  except:
    displaytype = []
  l_availableDisplayTypes.extend(displaytype)
unique_l_availableDisplayTypes = list(set(l_availableDisplayTypes))

feature_df['metadata_availableDisplayTypes'] = feature_df['metadata_clean'].apply(lambda x: x['availableDisplayTypes'] if 'availableDisplayTypes' in x.keys() else [] )

for i in unique_l_availableDisplayTypes:
  col_name = 'metadata_vailableDisplayTypes' + i
  print(col_name)
  feature_df[col_name] = feature_df['metadata_availableDisplayTypes'].apply(lambda x: 1 if i in x else 0)

feature_df = feature_df.drop(['metadata_availableDisplayTypes'], axis = 1)

metadata_vailableDisplayTypesmetadata_table
metadata_vailableDisplayTypesmap
metadata_vailableDisplayTypestable
metadata_vailableDisplayTypesfatrow
metadata_vailableDisplayTypesblob
metadata_vailableDisplayTypesdata_lens
metadata_vailableDisplayTypesvisualization
metadata_vailableDisplayTypespage
metadata_vailableDisplayTypesassetinventory
metadata_vailableDisplayTypeshref


In [22]:
feature_df = feature_df.drop(['metadata','metadata_clean','createYear','createMonth','rowsUpdatedAtYear','rowsUpdatedAtMonth'], axis = 1)

In [23]:
# Group rowsUpdatedBy
feature_df['rowsUpdatedBy'] = feature_df['rowsUpdatedBy'].fillna('Unknown')
rowsUpdatedBy_count = list(zip(feature_df['rowsUpdatedBy'].value_counts(),feature_df['rowsUpdatedBy'].value_counts().index))

s = 0
main_rowsUpdatedBy = []
for i in rowsUpdatedBy_count:
  s += i[0]
  main_rowsUpdatedBy.append(i[1])
  if s > feature_df.shape[0]*0.9:
    break
#print(len(main_rowsUpdatedBy)) #--> 8
# Group other categories into Other
feature_df['rowsUpdatedBy'] = feature_df['rowsUpdatedBy'].apply(lambda x: x if x in main_rowsUpdatedBy else 'Other' )



One-hot encoding on categorical features

In [24]:
df_encode = pd.get_dummies(feature_df.drop(['data_uuid'],axis = 1))
index = feature_df['data_uuid']
df_encode.head()

,newBackend,publicationAppendEnabled,createAtAge,flags_default,flags_ownerMayBeContacted,flags_restorable,flags_restorePossibleForType,tableAuthor_flags_acceptedEula,tableAuthor_flags_mayBeStoriesCoOwner,rowsUpdatedAtAge,metadata_UpdateFrequency_encode,metadata_vailableDisplayTypesmetadata_table,metadata_vailableDisplayTypesmap,metadata_vailableDisplayTypestable,metadata_vailableDisplayTypesfatrow,metadata_vailableDisplayTypesblob,metadata_vailableDisplayTypesdata_lens,metadata_vailableDisplayTypesvisualization,metadata_vailableDisplayTypespage,metadata_vailableDisplayTypesassetinventory,metadata_vailableDisplayTypeshref,assetType_chart,assetType_datalens,assetType_dataset,assetType_file,assetType_filter,assetType_href,assetType_map,assetType_visualization,category_Business,category_City Government,category_Education,category_Environment,category_Health,category_Housing & Development,category_NYC BigApps,category_Public Safety,category_Recreation,category_Social Services,category_Transportation,...,tableAuthor_screenName_Sudhir Vasudeva,tableAuthor_screenName_Tejas.Patel,tableAuthor_screenName_Theresa,tableAuthor_screenName_Vaughan Coleman,tableAuthor_screenName_kfontes,metadata_AgencyClean_311,metadata_AgencyClean_Brooklyn Borough President (BPBK),metadata_AgencyClean_Campaign Finance Board (CFB),metadata_AgencyClean_Department for the Aging (DFTA),metadata_AgencyClean_Department of Buildings (DOB),metadata_AgencyClean_Department of City Planning (DCP),metadata_AgencyClean_Department of Citywide Administrative Services (DCAS),metadata_AgencyClean_Department of Consumer Affairs (DCA),metadata_AgencyClean_Department of Education (DOE),metadata_AgencyClean_Department of Environmental Protection (DEP),metadata_AgencyClean_Department of Finance (DOF),metadata_AgencyClean_Department of Health and Mental Hygiene (DOHMH),metadata_AgencyClean_Department of Housing Preservation and Development (HPD),metadata_AgencyClean_Department of Information Technology & Telecommunications (DoITT),metadata_AgencyClean_Department of Parks and Recreation (DPR),metadata_AgencyClean_Department of Probation (DOP),metadata_AgencyClean_Department of Sanitation (DSNY),metadata_AgencyClean_Department of Small Business Services (SBS),metadata_AgencyClean_Department of Transportation (DOT),metadata_AgencyClean_Department of Youth and Community Development (DYCD),metadata_AgencyClean_Human Resources Administration (HRA),metadata_AgencyClean_Manhattan Borough President (MBPO),metadata_AgencyClean_Mayor's Office for Economic Opportunity,metadata_AgencyClean_Mayor's Office of Climate and Sustainability,metadata_AgencyClean_Mayor's Office of Contract Services (MOCS),metadata_AgencyClean_Mayor's Office of Management & Budget (OMB),metadata_AgencyClean_Mayor's Office of Operations (OPS),metadata_AgencyClean_Mayor's Office of the Chief Technology Officer (MOCTO),metadata_AgencyClean_Mayor’s Office to End Domestic and Gender-Based Violence,metadata_AgencyClean_New York City Housing Authority (NYCHA),metadata_AgencyClean_Office of Emergency Management (OEM),metadata_AgencyClean_Other,metadata_AgencyClean_Police Department (NYPD),metadata_AgencyClean_School Construction Authority (SCA),metadata_AgencyClean_Taxi and Limousine Commission (TLC)
0,True,False,1105,1,1,1,1,1,1,83,9,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,True,False,384,0,1,0,0,1,1,385,14,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,True,False,1285,1,1,1,1,0,1,1285,17,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,True,False,782,1,1,1,1,1,1,782,17,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,True,False,508,1,1,1,1,1,1,

In [25]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

#num_features = ['averageRating','downloadCount','numberOfComments','viewCount','createAtAge','rowsUpdatedAtAge']
num_features = ['createAtAge','rowsUpdatedAtAge']
df_encode[num_features] = ss.fit_transform(df_encode[num_features])
df_encode.head()

,newBackend,publicationAppendEnabled,createAtAge,flags_default,flags_ownerMayBeContacted,flags_restorable,flags_restorePossibleForType,tableAuthor_flags_acceptedEula,tableAuthor_flags_mayBeStoriesCoOwner,rowsUpdatedAtAge,metadata_UpdateFrequency_encode,metadata_vailableDisplayTypesmetadata_table,metadata_vailableDisplayTypesmap,metadata_vailableDisplayTypestable,metadata_vailableDisplayTypesfatrow,metadata_vailableDisplayTypesblob,metadata_vailableDisplayTypesdata_lens,metadata_vailableDisplayTypesvisualization,metadata_vailableDisplayTypespage,metadata_vailableDisplayTypesassetinventory,metadata_vailableDisplayTypeshref,assetType_chart,assetType_datalens,assetType_dataset,assetType_file,assetType_filter,assetType_href,assetType_map,assetType_visualization,category_Business,category_City Government,category_Education,category_Environment,category_Health,category_Housing & Development,category_NYC BigApps,category_Public Safety,category_Recreation,category_Social Services,category_Transportation,...,tableAuthor_screenName_Sudhir Vasudeva,tableAuthor_screenName_Tejas.Patel,tableAuthor_screenName_Theresa,tableAuthor_screenName_Vaughan Coleman,tableAuthor_screenName_kfontes,metadata_AgencyClean_311,metadata_AgencyClean_Brooklyn Borough President (BPBK),metadata_AgencyClean_Campaign Finance Board (CFB),metadata_AgencyClean_Department for the Aging (DFTA),metadata_AgencyClean_Department of Buildings (DOB),metadata_AgencyClean_Department of City Planning (DCP),metadata_AgencyClean_Department of Citywide Administrative Services (DCAS),metadata_AgencyClean_Department of Consumer Affairs (DCA),metadata_AgencyClean_Department of Education (DOE),metadata_AgencyClean_Department of Environmental Protection (DEP),metadata_AgencyClean_Department of Finance (DOF),metadata_AgencyClean_Department of Health and Mental Hygiene (DOHMH),metadata_AgencyClean_Department of Housing Preservation and Development (HPD),metadata_AgencyClean_Department of Information Technology & Telecommunications (DoITT),metadata_AgencyClean_Department of Parks and Recreation (DPR),metadata_AgencyClean_Department of Probation (DOP),metadata_AgencyClean_Department of Sanitation (DSNY),metadata_AgencyClean_Department of Small Business Services (SBS),metadata_AgencyClean_Department of Transportation (DOT),metadata_AgencyClean_Department of Youth and Community Development (DYCD),metadata_AgencyClean_Human Resources Administration (HRA),metadata_AgencyClean_Manhattan Borough President (MBPO),metadata_AgencyClean_Mayor's Office for Economic Opportunity,metadata_AgencyClean_Mayor's Office of Climate and Sustainability,metadata_AgencyClean_Mayor's Office of Contract Services (MOCS),metadata_AgencyClean_Mayor's Office of Management & Budget (OMB),metadata_AgencyClean_Mayor's Office of Operations (OPS),metadata_AgencyClean_Mayor's Office of the Chief Technology Officer (MOCTO),metadata_AgencyClean_Mayor’s Office to End Domestic and Gender-Based Violence,metadata_AgencyClean_New York City Housing Authority (NYCHA),metadata_AgencyClean_Office of Emergency Management (OEM),metadata_AgencyClean_Other,metadata_AgencyClean_Police Department (NYPD),metadata_AgencyClean_School Construction Authority (SCA),metadata_AgencyClean_Taxi and Limousine Commission (TLC)
0,True,False,-0.486769,1,1,1,1,1,1,-0.855758,9,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,True,False,-1.153617,0,1,0,0,1,1,-0.529072,14,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,True,False,-0.320288,1,1,1,1,0,1,0.444494,17,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,True,False,-0.785509,1,1,1,1,1,1,-0.099621,17,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Compute Simialrity

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(df_encode)
print('pairwise dense output:\n {}\n'.format(similarities))

pairwise dense output:
 [[1.         0.93047832 0.94735675 ... 0.90502415 0.92108827 0.89246766]
 [0.93047832 1.         0.95726546 ... 0.91985737 0.93024143 0.93883761]
 [0.94735675 0.95726546 1.         ... 0.97611211 0.97748258 0.95115782]
 ...
 [0.90502415 0.91985737 0.97611211 ... 1.         0.99368778 0.93986026]
 [0.92108827 0.93024143 0.97748258 ... 0.99368778 1.         0.93881829]
 [0.89246766 0.93883761 0.95115782 ... 0.93986026 0.93881829 1.        ]]



In [ ]:
# function for recommendations
def spatial_recommendation(similarities_matrix, dataset_index, num_recommendation = 5):
  print('Input dataset index: ', dataset_index, '\nName: ',metadata_df['name'][dataset_index])
  index_lst = sorted(range(len(similarities_matrix[dataset_index])), key=lambda i: similarities_matrix[dataset_index][i], reverse = True)[1:num_recommendation+1]
  print('\nRecomendations:')
  for i in index_lst:
    print(metadata_df['name'][i])

  return index_lst, metadata_df['name'][index_lst].values

In [ ]:
#idx_lst, dataset_name = spatial_recommendation(similarities,3, 5)

Compute tag distance

In [27]:
# Extract tag information
feature_df['tags_clean'] = metadata_df['tags'].fillna('[]')
feature_df['tags_clean'] = feature_df['tags_clean'].apply(lambda x: ast.literal_eval(x))
l_tags = []
for i in feature_df['tags_clean']:
  #t = ast.literal_eval(i)
  if type(i) == list:
    l_tags.extend(i)
  else:
    l_tags.append(i)
unique_l_tags = list(set(l_tags))
print('The datasets have', len(unique_l_tags), 'unique tags')

tag_dict = {}
for i in unique_l_tags:
  tag_dict[i] = 0

for i in feature_df['tags_clean']:
  if type(i) == list and len(i) >0:
    for j in i:
      tag_dict[j] += 1
  else:
    if type(i) == str:
      tag_dict[i] += 1

sort_tag_dict = {k: v for k, v in sorted(tag_dict.items(), key=lambda item: item[1], reverse = True)}
#print(sort_tag_dict)

The datasets have 3616 unique tags


In [28]:
tags_df = feature_df.drop(feature_df.columns.difference(['data_uuid','tags_clean']), 1)
for t in unique_l_tags:
  tags_df[t] = tags_df['tags_clean'].apply(lambda x: 1 if t in x else 0)

tags_df = tags_df.drop(['data_uuid','tags_clean'], axis = 1)

In [29]:
from sklearn.metrics.pairwise import manhattan_distances
#tag_distance_matrix = manhattan_distances(tags_df)
#tag_distance_matrix

tag_similarity_matrix = cosine_similarity(tags_df) # manhattan_distances(tags_df)
tag_similarity_matrix


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [30]:
tag_similarity_matrix[0]

array([1., 0., 0., ..., 0., 0., 0.])

In [31]:
tag_np = np.array(tags_df)

In [32]:
import tensorflow as tf
tag_common_matrix = tf.matmul(tag_np, tag_np.T)


In [33]:
tag_common_matrix_array = np.array(tag_common_matrix)

In [ ]:
def combine_recommendation(similarities_matrix, tag_distance_matrix, dataset_index, num_recommendation = 5, spatial_weight=0.5):
  rank_df = feature_df.drop(feature_df.columns.difference(['data_uuid']), 1)
  rank_df['name'] = metadata_df['name']
  print('Input dataset index: ', dataset_index, '\nName: ',metadata_df['name'][dataset_index])
  #index_lst_spatial = sorted(range(len(similarities_matrix[dataset_index])), key=lambda i: similarities_matrix[dataset_index][i], reverse = True)
  #rank_spatial = {key: rank for rank, key in enumerate(index_lst)}
  rank_df['spatial_distance'] = similarities_matrix[dataset_index]
  rank_df['spatial_rank'] = rank_df['spatial_distance'].rank(method='min', ascending=False)
  # Tag
  rank_df['tag_distance'] =  tag_distance_matrix[dataset_index]
  rank_df['tag_rank'] = rank_df['tag_distance'].rank(method='min', ascending=True)
  # Weighted rank
  rank_df['weighted_rank'] = (1-spatial_weight)*rank_df['tag_rank'] + spatial_weight*rank_df['spatial_rank']
  rank_df_sorted = rank_df.sort_values(by=['weighted_rank'],inplace = False)
  rank_df_sorted = rank_df_sorted.reset_index(drop= True)
  print('\nRecomendations:')
  for i in range(1,num_recommendation+1):
    print(rank_df_sorted['name'][i])

  return rank_df_sorted

Combine with column embedding

In [34]:
import ast
import numpy as np
def from_np_array(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.array(ast.literal_eval(array_string))

dataset_embedding_df = pd.read_csv('/content/drive/MyDrive/metavis/dataset_embedding_v20211117.csv', converters={'dataset_embedding': from_np_array})

In [35]:
dataset_embedding_df.head()

,data_uuid,dataset_embedding
0,2232-dj5q,"[0.272886902, 0.651672661, 0.164423525, 0.0848..."
1,22gm-5ceg,"[-0.03767802, -0.07734199, -0.1406482, -0.0937..."
2,22rr-ujq3,"[-0.09199004, -0.09967875, 0.00158188, -0.0572..."
3,23z9-6uk9,"[0.11498749, 0.33656264, 0.27530958, 0.0716580..."
4,242c-ru4i,"[-0.145268378, 0.0265072141, 0.456168638, -0.2..."


In [36]:
dataset_embedding_df_transform = pd.DataFrame(dataset_embedding_df['dataset_embedding'].tolist())

In [37]:
embedding_similarities = cosine_similarity(dataset_embedding_df_transform)
print('pairwise dense output:\n {}\n'.format(embedding_similarities))

pairwise dense output:
 [[1.         0.52378347 0.56785014 ... 0.72235943 0.76393191 0.36181743]
 [0.52378347 1.         0.49350779 ... 0.51760571 0.51444995 0.50656185]
 [0.56785014 0.49350779 1.         ... 0.52054206 0.56296838 0.25090645]
 ...
 [0.72235943 0.51760571 0.52054206 ... 1.         0.76859075 0.33655124]
 [0.76393191 0.51444995 0.56296838 ... 0.76859075 1.         0.32363104]
 [0.36181743 0.50656185 0.25090645 ... 0.33655124 0.32363104 1.        ]]



Combine with LDA topics

In [38]:
lda_df = pd.read_csv('/content/drive/MyDrive/metavis/topic_result_spark.csv')
lda_df_filter = lda_df.drop(['name','description_processed','category'], axis = 1)
lda_df_filter.head()

,data_uuid,description,topic
0,2232-dj5q,Data on the youth who are currently residing i...,3
1,22gm-5ceg,GIS shapefiles showing the areas in NYC where ...,7
2,22rr-ujq3,Student discipline annual report for English l...,10
3,23z9-6uk9,Directory of Department of Education High Scho...,0
4,242c-ru4i,Location of DSNY Special Waste Drop-Off Sites....,3


In [43]:
lda_df_old = pd.read_csv('/content/drive/MyDrive/metavis/topic_result.csv')
lda_df_filter_old = lda_df_old.drop(['name','description_processed','category'], axis = 1)
lda_df_filter_old.head()

,data_uuid,description,topic
0,2232-dj5q,Data on the youth who are currently residing i...,0
1,22gm-5ceg,GIS shapefiles showing the areas in NYC where ...,8
2,22rr-ujq3,Student discipline annual report for English l...,1
3,23z9-6uk9,Directory of Department of Education High Scho...,1
4,242c-ru4i,Location of DSNY Special Waste Drop-Off Sites....,1


In [39]:
index_uuid_df = pd.DataFrame(metadata_df[['data_uuid','name']])
index_uuid_df = index_uuid_df.reset_index()
index_uuid_df.head()

,index,data_uuid,name
0,0,2232-dj5q,Local Law 37 - DYCD Report
1,1,22gm-5ceg,Shapefiles of BOA and Community Brownfield Pla...
2,2,22rr-ujq3,2016-2017 Student Discipline Annual Report - ELL
3,3,23z9-6uk9,2020 DOE High School Directory
4,4,242c-ru4i,DSNY Special Waste Drop-off Sites


In [84]:
# output csv file
index_uuid_df.to_csv("index_uuid_df.csv", index = False)

In [94]:
pd.DataFrame(similarities).to_csv("/content/drive/MyDrive/metavis/similarities.csv")

In [91]:
pd.DataFrame(tag_common_matrix_array).to_csv("tag_common_matrix_array.csv")

In [92]:
lda_df_filter_old.to_csv("lda_df_filter.csv", index = False)

In [99]:
pd.DataFrame(embedding_similarities).to_csv("/content/drive/MyDrive/metavis/embedding_similaritie.csv")

### Function

In [96]:
def lda_spatial_recommendation(similarities_matrix, tag_distance_matrix, dataset_index, lda_df, index_uuid_df,  embedding_similarities, num_recommendation = 5, 
                               spatial_weight=0.5, embedding_weight =0.4, semantic = True):
  rank_df = index_uuid_df.drop(index_uuid_df.columns.difference(['data_uuid']), 1)
  rank_df['name'] = index_uuid_df['name']
  print('Input dataset index: ', dataset_index, '\nName: ',index_uuid_df['name'][dataset_index])
  rank_df['spatial_similarity'] = similarities_matrix[dataset_index]
  rank_df['spatial_rank'] = rank_df['spatial_similarity'].rank(method='min', ascending=False)
  # Tag
  rank_df['tag_common'] =  tag_distance_matrix[dataset_index]
  rank_df['tag_rank'] = rank_df['tag_common'].rank(method='min', ascending=False) #rank_df['tag_distance'].rank(method='min', ascending=True)
  # Embedding
  
  rank_df['embedding_similarity'] = embedding_similarities[dataset_index]
  rank_df['embedding_rank'] = rank_df['embedding_similarity'].rank(method='min', ascending=False)

  # Weighted rank
  rank_df['weighted_rank'] = (1-spatial_weight-embedding_weight)*rank_df['tag_rank'] + spatial_weight*rank_df['spatial_rank'] + embedding_weight*rank_df['embedding_rank']
  rank_df_sorted = rank_df.sort_values(by=['weighted_rank'],inplace = False)
  rank_df_sorted = rank_df_sorted.reset_index(drop= True)
  rank_df_sorted_merge = rank_df_sorted.merge(lda_df, how='inner', on='data_uuid')
  
  if semantic:
    data_topic = rank_df_sorted_merge['topic'][0]
    rank_df_sorted_merge = pd.DataFrame(rank_df_sorted_merge[rank_df_sorted_merge['topic'] == data_topic])

  print('\nRecomendations:')
  for i in range(1,num_recommendation+1):
    print(list(rank_df_sorted_merge['name'])[i])

  return rank_df_sorted_merge

Giving Recommendations..
193
199
For presentation use

In [ ]:
recommendation = recommendation_dataset('Street and Highway Capital Reconstruction Projects - Intersection (Map)',
                                            num_recommendation = 5, metadata_weight=0.2, word_embedding_weight =0.4, use_lda = False)

Input dataset index:  199 
Name:  Street and Highway Capital Reconstruction Projects - Intersection (Map)

Recomendations:
Street and Highway Capital Reconstruction Projects - Block (Map)
Open Street Locations (Map)
Housing Database by City Council (Map)
Housing Database by Community District (Map)
Pedestrian Ramp Locations (Map)


In [ ]:
recommendation = recommendation_dataset('Street and Highway Capital Reconstruction Projects - Intersection (Map)',
                                             num_recommendation = 5, metadata_weight=0.2, word_embedding_weight =0.4, use_lda = True)

Input dataset index:  199 
Name:  Street and Highway Capital Reconstruction Projects - Intersection (Map)

Recomendations:
Street and Highway Capital Reconstruction Projects - Block (Map)
Open Street Locations (Map)
New York City Truck Routes - Map
Street Pothole Work Orders - Closed (Map)
Outdoor Learning - Streets Locations Map (Historical)


### Recommendations...

In [97]:
recommendation = lda_spatial_recommendation(similarities, tag_common_matrix_array, 199, lda_df_filter, index_uuid_df, embedding_similarities,
                                            num_recommendation = 5, spatial_weight=0.2, embedding_weight =0.4, semantic = False)
recommendation.head(6)

Input dataset index:  199 
Name:  Street and Highway Capital Reconstruction Projects - Intersection (Map)

Recomendations:
Street and Highway Capital Reconstruction Projects - Block (Map)
Open Street Locations (Map)
Housing Database by City Council (Map)
Housing Database by Community District (Map)
Pedestrian Ramp Locations (Map)


,data_uuid,name,spatial_similarity,spatial_rank,tag_common,tag_rank,embedding_similarity,embedding_rank,weighted_rank,description,topic
0,3zy2-a8eg,Street and Highway Capital Reconstruction Proj...,1.000000,1.0,6,1.0,1.000000,1.0,1.0,This data is a spatial representation of stree...,4
1,si9g-fztb,Street and Highway Capital Reconstruction Proj...,1.000000,1.0,6,1.0,0.980236,2.0,1.4,This data is a spatial representation of stree...,4
2,6q9w-6svb,Open Street Locations (Map),0.988428,10.0,1,8.0,0.823009,14.0,10.8,New York City is opening streets daily for ped...,4
3,gf2p-d6pi,Housing Database by City Council (Map),0.976460,21.0,1,8.0,0.797212,30.0,19.4,Net change in housing units arising from new b...,11
4,m8qf-ycd5,Housing Database by Community District (Map),0.976460,21.0,1,8.0,0.778834,46.0,25.8,Net change in housing units arising from new b...,11
5,u7ws-2dus,Pedestrian Ramp Locations (Map),0.989071,8.0,1,8.0,0.764915,70.0,32.8,Pedestrian ramps provide access on and off str...,4


In [98]:
recommendation = lda_spatial_recommendation(similarities, tag_common_matrix_array, 199, lda_df_filter, index_uuid_df, embedding_similarities, num_recommendation = 5, 
                                            spatial_weight=0.2, embedding_weight =0.4, semantic = True)
recommendation.head(6)

Input dataset index:  199 
Name:  Street and Highway Capital Reconstruction Projects - Intersection (Map)

Recomendations:
Street and Highway Capital Reconstruction Projects - Block (Map)
Open Street Locations (Map)
Pedestrian Ramp Locations (Map)
Bus Lanes - Local Streets (Map)
New York City Truck Routes - Map


,data_uuid,name,spatial_similarity,spatial_rank,tag_common,tag_rank,embedding_similarity,embedding_rank,weighted_rank,description,topic
0,3zy2-a8eg,Street and Highway Capital Reconstruction Proj...,1.000000,1.0,6,1.0,1.000000,1.0,1.0,This data is a spatial representation of stree...,4
1,si9g-fztb,Street and Highway Capital Reconstruction Proj...,1.000000,1.0,6,1.0,0.980236,2.0,1.4,This data is a spatial representation of stree...,4
2,6q9w-6svb,Open Street Locations (Map),0.988428,10.0,1,8.0,0.823009,14.0,10.8,New York City is opening streets daily for ped...,4
5,u7ws-2dus,Pedestrian Ramp Locations (Map),0.989071,8.0,1,8.0,0.764915,70.0,32.8,Pedestrian ramps provide access on and off str...,4
6,rx8t-6euq,Bus Lanes - Local Streets (Map),0.988167,12.0,0,89.0,0.848044,3.0,39.2,Bus lanes are travel lanes that are restricted...,4
7,wnu3-egq7,New York City Truck Routes - Map,0.988722,9.0,0,89.0,0.837958,5.0,39.4,The New York City Truck Route Network is a set...,4


Test

In [82]:
recommendation = lda_spatial_recommendation(similarities, tag_common_matrix_array, 2500, lda_df_filter_old, index_uuid_df, num_recommendation = 5, 
                                            spatial_weight=0.2, embedding_weight =0.4, semantic = True)
recommendation.head(6)

Input dataset index:  2500 
Name:  LinkNYC Kiosk Status (Historical)

Recomendations:
LinkNYC Kiosk Status
DOB Certificate Of Occupancy
Local Law 8 of 2020 – Complaints of Illegal Parking of Vehicles Operated on Behalf of the City
Legally Operating Businesses
Trade Waste Broker Registrants


,data_uuid,name,spatial_similarity,spatial_rank,tag_common,tag_rank,embedding_similarity,embedding_rank,weighted_rank,description,topic
0,quxm-hmyr,LinkNYC Kiosk Status (Historical),1.000000,1.0,11,1.0,1.000000,1.0,1.0,This dataset provides the most current listing...,0
1,n6c5-95xh,LinkNYC Kiosk Status,1.000000,1.0,11,1.0,0.999508,2.0,1.4,This dataset provides the most current listing...,0
2,bs8b-p36w,DOB Certificate Of Occupancy,0.788145,40.0,0,64.0,0.965891,45.0,51.6,A Certificate of Occupancy (CO) states a build...,0
4,cwy2-px8b,Local Law 8 of 2020 – Complaints of Illegal Pa...,0.832292,7.0,0,64.0,0.961029,81.0,59.4,<b>NOTE: This data does not present a full pic...,0
5,w7w3-xahh,Legally Operating Businesses,0.673525,151.0,0,64.0,0.972595,16.0,62.2,This data set features businesses/individuals ...,0
8,krx7-u82t,Trade Waste Broker Registrants,0.824009,9.0,0,64.0,0.951112,121.0,75.8,Companies listed as of the most recent export ...,0


In [83]:
recommendation = lda_spatial_recommendation(similarities, tag_common_matrix_array, 2500, lda_df_filter, index_uuid_df, num_recommendation = 5, 
                                            spatial_weight=0.2, embedding_weight =0.4, semantic = True)
recommendation.head(6)

Input dataset index:  2500 
Name:  LinkNYC Kiosk Status (Historical)

Recomendations:
LinkNYC Kiosk Status
DOB Certificate Of Occupancy
Denied TW and Wholesale Market Companies
Trade Waste Broker Registrants
Self Hauler Registrants


,data_uuid,name,spatial_similarity,spatial_rank,tag_common,tag_rank,embedding_similarity,embedding_rank,weighted_rank,description,topic
0,quxm-hmyr,LinkNYC Kiosk Status (Historical),1.000000,1.0,11,1.0,1.000000,1.0,1.0,This dataset provides the most current listing...,4
1,n6c5-95xh,LinkNYC Kiosk Status,1.000000,1.0,11,1.0,0.999508,2.0,1.4,This dataset provides the most current listing...,4
2,bs8b-p36w,DOB Certificate Of Occupancy,0.788145,40.0,0,64.0,0.965891,45.0,51.6,A Certificate of Occupancy (CO) states a build...,4
3,exsg-kpya,Denied TW and Wholesale Market Companies,0.794226,31.0,0,64.0,0.965316,57.0,54.6,Denied TW and Wholesale Market Companies datas...,4
8,krx7-u82t,Trade Waste Broker Registrants,0.824009,9.0,0,64.0,0.951112,121.0,75.8,Companies listed as of the most recent export ...,4
9,a8wp-rerh,Self Hauler Registrants,0.824004,10.0,0,64.0,0.951112,121.0,76.0,Companies listed as of the most recent export ...,4


In [ ]:
def connection(similarities_matrix, tag_distance_matrix, dataset_index, lda_df, index_uuid_df, 
               num_recommendation = 5, spatial_weight=0.5, embedding_weight =0.4, semantic = True):
  
    base_recommendation = lda_spatial_recommendation(similarities_matrix, tag_distance_matrix, dataset_index, lda_df, index_uuid_df, num_recommendation, spatial_weight, embedding_weight, semantic)
    base_recommendation_uuid = base_recommendation['data_uuid'][:10].to_list()
    
    target_lst =  base_recommendation['data_uuid'][:10].to_list()
    dataset_uuid = index_uuid_df[index_uuid_df['index'] == dataset_index]['data_uuid'].to_list()[0]
    source_lst = [dataset_uuid]*10

    for i in base_recommendation_uuid:
      new_dataset_index = index_uuid_df[index_uuid_df['data_uuid'] == i]['index'].to_list()[0]
      recom = lda_spatial_recommendation(similarities_matrix, tag_distance_matrix, new_dataset_index, lda_df, index_uuid_df, num_recommendation, spatial_weight, embedding_weight, semantic)
      new_recom_uuid =  recom['data_uuid'][:10].to_list()
      for j in new_recom_uuid:
        if j in base_recommendation_uuid:
          source_lst.append(i)
          target_lst.append(j)
    print(source_lst)
    print(target_lst)
    connection_df = pd.DataFrame(zip(source_lst, target_lst), columns = ['source', 'target'])
    connection_df = connection_df[connection_df['source'] != connection_df['target']]
    return connection_df

 


In [ ]:
a = index_uuid_df[index_uuid_df['index'] == 199]['data_uuid']

In [ ]:
connection(similarities, tag_common_matrix_array, 199, lda_df_filter, index_uuid_df, num_recommendation = 5, 
                                            spatial_weight=0.2, embedding_weight =0.4, semantic = True)

Input dataset index:  199 
Name:  Street and Highway Capital Reconstruction Projects - Intersection (Map)
Input dataset index:  199 
Name:  Street and Highway Capital Reconstruction Projects - Intersection (Map)
Input dataset index:  2662 
Name:  Street and Highway Capital Reconstruction Projects - Block (Map)
Input dataset index:  485 
Name:  Open Street Locations (Map)
Input dataset index:  3132 
Name:  New York City Truck Routes - Map
Input dataset index:  1350 
Name:  Street Pothole Work Orders - Closed (Map)
Input dataset index:  2831 
Name:  Outdoor Learning - Streets Locations Map (Historical)
Input dataset index:  877 
Name:  DSNY Planned Commercial Waste Zones (Map)
Input dataset index:  3035 
Name:  Outdoor Learning - Streets Locations (Historical)
Input dataset index:  363 
Name:  Pre-Qualified List
Input dataset index:  939 
Name:  Street Network Changes
['3zy2-a8eg', '3zy2-a8eg', '3zy2-a8eg', '3zy2-a8eg', '3zy2-a8eg', '3zy2-a8eg', '3zy2-a8eg', '3zy2-a8eg', '3zy2-a8eg', '3z

,source,target
1,3zy2-a8eg,si9g-fztb
2,3zy2-a8eg,6q9w-6svb
3,3zy2-a8eg,wnu3-egq7
4,3zy2-a8eg,emex-8upu
5,3zy2-a8eg,u3t3-74n4
6,3zy2-a8eg,a7bv-5698
7,3zy2-a8eg,vtjm-ngnu
8,3zy2-a8eg,5hsa-dfq5
9,3zy2-a8eg,as9z-kwsh
11,3zy2-a8eg,si9g-fztb


End

In [ ]:
feature_df[feature_df['data_uuid']=='bvzk-auuz']

,data_uuid,assetType,averageRating,category,displayType,downloadCount,newBackend,numberOfComments,provenance,publicationAppendEnabled,rowsUpdatedBy,viewCount,viewType,createAtAge,flags_default,flags_ownerMayBeContacted,flags_restorable,flags_restorePossibleForType,tableAuthor_screenName,tableAuthor_flags_acceptedEula,tableAuthor_flags_mayBeStoriesCoOwner,rowsUpdatedAtAge,metadata_AgencyClean,metadata_UpdateFrequency_encode,metadata_vailableDisplayTypestable,metadata_vailableDisplayTypespage,metadata_vailableDisplayTypesassetinventory,metadata_vailableDisplayTypesvisualization,metadata_vailableDisplayTypesdata_lens,metadata_vailableDisplayTypesmetadata_table,metadata_vailableDisplayTypesfatrow,metadata_vailableDisplayTypesblob,metadata_vailableDisplayTypeshref,metadata_vailableDisplayTypesmap,tags_clean
1054,bvzk-auuz,dataset,0,Education,table,365,True,0,official,False,ftn3-awiu,1284,tabular,1295,1,1,1,1,Vaughan Coleman,1,1,1295,Department of Education (DOE),17,1,1,0,0,0,0,1,0,0,0,"[doe, demographic snapshot]"


In [ ]:
feature_df[feature_df['data_uuid']=='3w89-mz4u']

,data_uuid,assetType,averageRating,category,displayType,downloadCount,newBackend,numberOfComments,provenance,publicationAppendEnabled,rowsUpdatedBy,viewCount,viewType,createAtAge,flags_default,flags_ownerMayBeContacted,flags_restorable,flags_restorePossibleForType,tableAuthor_screenName,tableAuthor_flags_acceptedEula,tableAuthor_flags_mayBeStoriesCoOwner,rowsUpdatedAtAge,metadata_AgencyClean,metadata_UpdateFrequency_encode,metadata_vailableDisplayTypestable,metadata_vailableDisplayTypespage,metadata_vailableDisplayTypesassetinventory,metadata_vailableDisplayTypesvisualization,metadata_vailableDisplayTypesdata_lens,metadata_vailableDisplayTypesmetadata_table,metadata_vailableDisplayTypesfatrow,metadata_vailableDisplayTypesblob,metadata_vailableDisplayTypeshref,metadata_vailableDisplayTypesmap,tags_clean
189,3w89-mz4u,dataset,0,Education,table,377,True,0,official,False,ftn3-awiu,2783,tabular,1307,1,1,1,1,Vaughan Coleman,1,1,1307,Department of Education (DOE),17,1,1,0,0,0,0,1,0,0,0,"[doe, class size, k8]"


In [ ]:

feature_df[feature_df['data_uuid']=='6yzk-rwz2']

,data_uuid,assetType,category,displayType,newBackend,provenance,publicationAppendEnabled,rowsUpdatedBy,viewType,createAtAge,flags_default,flags_ownerMayBeContacted,flags_restorable,flags_restorePossibleForType,tableAuthor_screenName,tableAuthor_flags_acceptedEula,tableAuthor_flags_mayBeStoriesCoOwner,rowsUpdatedAtAge,metadata_AgencyClean,metadata_UpdateFrequency_encode,metadata_vailableDisplayTypestable,metadata_vailableDisplayTypespage,metadata_vailableDisplayTypesassetinventory,metadata_vailableDisplayTypesvisualization,metadata_vailableDisplayTypesdata_lens,metadata_vailableDisplayTypesmetadata_table,metadata_vailableDisplayTypesfatrow,metadata_vailableDisplayTypesblob,metadata_vailableDisplayTypeshref,metadata_vailableDisplayTypesmap,tags_clean
519,6yzk-rwz2,dataset,City Government,table,True,official,False,5fuc-pqz2,tabular,709,1,1,1,1,NYC OpenData,1,1,706,Mayor's Office of the Chief Technology Officer...,14,1,1,0,0,0,0,1,0,0,0,"[internet master plan, internet, broadband, br..."


In [ ]:
from pprint import pprint
pprint(list(lda_df[lda_df['data_uuid']=='3w3r-v568']['description'])[0])

'JFK Airtrain'


In [ ]:
from pprint import pprint
pprint(list(lda_df[lda_df['data_uuid']=='3vjv-6tf5']['description'])[0])

('Directory of Pools\r\n'
 '\r\n'
 'This dataset consists of indoor and outdoor pools under the jurisdiction of '
 'NYC Parks. For more information about NYC Parks pools, visit '
 'https://www.nycgovparks.org/highlights/places-to-go/pools\r\n'
 '\r\n'
 'This dataset uses the standard NYC projection of '
 'NAD_1983_StatePlane_New_York_Long_Island_FIPS_3104_Feet. Lengths are in feet '
 'and areas in square feet.')


In [ ]:
pprint(lda_df['name'][180:200])

180    Verified Locations for NYC City-Funded Social ...
181     2006-2012 Math Test Results - School - Ethnicity
182    Community Development Block Grant (CDBG) Eligi...
183                          NYCHA Residential Addresses
184                                Small Purchase Report
185                                 2050s Mean Sea Level
186                                                Pools
187                           Tax Liability By AGI Range
188                                         JFK Airtrain
189    2017-2018 Class Size Report Borough K-8 Averag...
190                   2005 - 2020 Quality Review Ratings
191           Compliance at-the-tap Lead and Copper Data
192    2011-2012 School Closure Discharge Reporting -...
193         2013-2019 Math Test Results Charter - School
194                  Watershed Water Quality - Limnology
195      2015-2018 Historical Daily Attendance By School
196              2018-2019 Citywide Demographic Snapshot
197    Universal Solicitation o